# Resnet

In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Cargar el modelo ResNet50 preentrenado
model = models.resnet50(pretrained=True)

# Eliminar la capa de clasificación final para obtener solo los embeddings
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Asegurarse de que el modelo está en modo de evaluación
model.eval()

# Definir las transformaciones para preprocesar la imagen
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Cargar y preprocesar la imagen
img_path = "perritos.jpg"  # Cambia por tu imagen
img = Image.open(img_path)
img_tensor = preprocess(img).unsqueeze(0)  # Añadir la dimensión de batch

# Extraer los embeddings de la imagen
with torch.no_grad():
    embeddings = model(img_tensor)

# Convertir los embeddings a un formato numpy
embeddings_np = embeddings.squeeze().numpy()

print("Embeddings de la imagen:", embeddings_np.shape)

C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Embeddings de la imagen: (2048,)


In [2]:
embeddings_np 

array([0.3880407 , 0.9367521 , 1.1423601 , ..., 0.43771523, 0.12080878,
       0.25377283], dtype=float32)

# CLIP

CLIP puede trabajar tanto con imágenes como con texto, lo que lo hace ideal para tareas donde es necesario asociar imágenes con descripciones o trabajar con ambos tipos de datos simultáneamente.

In [3]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Cargar el modelo y el procesador CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Cargar y preprocesar la imagen
img_path = "perritos.jpg"
image = Image.open(img_path)
inputs = processor(images=image, return_tensors="pt")

# Extraer los embeddings de la imagen
with torch.no_grad():
    embeddings = model.get_image_features(**inputs)

# Convertir los embeddings a numpy
embeddings_np = embeddings.squeeze().numpy()

print("Embeddings de la imagen:", embeddings_np.shape)


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USUARIO\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Embeddings de la imagen: (512,)


In [4]:
# Cargar el modelo y el procesador CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Cargar y preprocesar la imagen
img_path = "perritos.jpg"
image = Image.open(img_path)
inputs = processor(images=image, return_tensors="pt")

# Extraer los embeddings de la imagen
with torch.no_grad():
    embeddings = model.get_image_features(**inputs)

# Convertir los embeddings a numpy
embeddings_np = embeddings.squeeze().numpy()

print("Embeddings de la imagen:", embeddings_np.shape)

Embeddings de la imagen: (512,)


In [6]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch


In [1]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

# Cargar el modelo CLIP y el procesador
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Imagen a analizar
image = Image.open("perritos.jpg")

# Descripciones textuales para comparar
text = ["Un gato blanco durmiendo en el sofá", "Una persona paseando varios perros", "un carro estacionado"]

# Preprocesar imagen y texto
inputs = processor(text=text, images=image, return_tensors="pt", padding=True)

# Separar los inputs en características para imagen y texto
image_inputs = inputs['pixel_values']  # Características de la imagen
text_inputs = inputs['input_ids']  # Características del texto
attention_mask = inputs['attention_mask']  # Máscara de atención para el texto

# Obtener las características de la imagen y el texto
with torch.no_grad():
    image_features = model.get_image_features(pixel_values=image_inputs)  # Solo pasar las características de la imagen
    text_features = model.get_text_features(input_ids=text_inputs, attention_mask=attention_mask)  # Solo texto

# Calcular la similitud entre la imagen y los textos
logits_per_image = image_features @ text_features.T  # Producto punto entre características de imagen y texto
probs = logits_per_image.softmax(dim=1)  # Convertir en probabilidades

print("Probabilidades de similitud imagen-texto:", probs)


Probabilidades de similitud imagen-texto: tensor([[2.1143e-08, 1.0000e+00, 1.9094e-06]])


In [2]:
for i, prob in enumerate(probs[0]):
    print(f"Probabilidad de similitud para texto '{text[i]}': {round(prob.item(), 3)}")

Probabilidad de similitud para texto 'Un gato blanco durmiendo en el sofá': 0.0
Probabilidad de similitud para texto 'Una persona paseando varios perros': 1.0
Probabilidad de similitud para texto 'un carro estacionado': 0.0


probs

In [13]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# Cargar el modelo BLIP y el procesador
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Cargar la imagen
image = Image.open("perritos.jpg")

# Preprocesar la imagen
inputs = processor(images=image, return_tensors="pt")

# Generar subtítulo
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)

print("Subtítulo generado:", caption)
    

C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Subtítulo generado: a woman walking her dogs down the street
